In [ ]:
# 0) bootstrap
import os, sys
from pathlib import Path
ROOT = Path.cwd()
while not (ROOT / "pyproject.toml").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent
os.chdir(ROOT)
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
print("Project root:", ROOT)

In [ ]:
# 1) choose doc
doc_id = "NFS_2019"
mined_dir = Path("data/mined") / doc_id

In [ ]:
# 2) build specs from mined outputs
from packages.calculators.parser import build_specs_from_mined
specs = build_specs_from_mined(doc_id, mined_dir=mined_dir, prefer_latex=True, max_specs=10)
len(specs), specs[:2]

In [ ]:
# 3) register all specs
from packages.calculators.registry import register, list_tools
for s in specs:
    register(s)

tools = list_tools(doc_id)
len(tools)

In [ ]:
# 4) pick one tool to execute
from packages.calculators.executor import evaluate
import json

if tools:
    demo = tools[0]
    print("Picked tool:", demo["name"], demo["expression"])
    # Create a dummy input dict; tweak keys to match demo["inputs"]
    inputs = {}
    for v in demo.get("inputs", []):
        # naive defaults: '1' or a unit-aware string if unit present
        inputs[v["name"]] = f"1 {v['unit']}" if v.get("unit") else 1
    out = evaluate(
        # rebuild ToolSpec (tiny bridge)
        __import__("packages.calculators.parser", fromlist=["ToolSpec"]).ToolSpec(
            id=demo["id"],
            name=demo["name"],
            doc_id=demo["doc_id"],
            version=demo.get("version",1),
            expression=demo["expression"],
            target=demo.get("target"),
            inputs=[__import__("packages.calculators.parser", fromlist=["ToolVar"]).ToolVar(**iv) for iv in demo.get("inputs",[])],
            notes=demo.get("notes",""),
            provenance=demo.get("provenance",{}),
        ),
        inputs=inputs
    )
    print("Result:", json.dumps(out, indent=2))
else:
    print("No tools built. Check mined JSONLs or run the miner on pages with formulas.")